Este script sirve para: <br>
1.- Abrir el navegador<br>
2.- Ir a ALICIA (https://alicia.concytec.gob.pe/)<br>
3.- Lanzar búsquedas usando términos específicos<br>
4.- Extraer los metadatos de todos los resultados<br>
5.- Guardar los metadatos en un archivo externo

# 1. Librerías

In [1]:
import pandas as pd
import time, os, re

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 2. Variables globales

In [3]:
ALICIA_url = 'https://alicia.concytec.gob.pe'

In [4]:
#esta línea debe actualizarse si se cambia de versión de chromedriver
chrome_path = r'.\ext\111.0.5563.64\chromedriver.exe'

# 3. Definición de funciones (I)

In [5]:
#Función para iniciar una nueva instancia del navegador Chrome
def openWebNav():
    
    WebNavObj = webdriver.Chrome(executable_path = chrome_path)
    
    return WebNavObj

In [6]:
def getCurrentHTML(WebNavObj):
    ps = WebNavObj.page_source
    soupObj =BeautifulSoup(ps, 'lxml')
    
    return soupObj

In [7]:
def runALICIAsearch(WebNavObj, kwStr):
    selStr = "input[type='text']"

    try:
        sBox = WebDriverWait(WebNavObj,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selStr)))
        sBox.clear()
        sBox.send_keys(kwStr + '\n')
    except:
        print(f"\tError al buscar {kwStr}. Hazlo manualmente...")
        input("\tEn espera...(presione tecla)...")

In [8]:
def getNrALICIA(WebNavObj):
    soupObj = getCurrentHTML(WebNavObj)
    tagNr = ['search-stats', 'strong']
    
    NrElem = soupObj.find(class_=tagNr[0])
    
    if NrElem is not None:
        try:
            Nr = NrElem.find_all(tagNr[1])
            Nr = Nr[1]
            Nr = Nr.text.replace(",","")
            Nr = int(Nr)
        except:
            print("No se ubicó la cantidad de resultados.")
            Nr = input("Ingrese el número manualmente =>")
            Nr = int(Nr)
    else:
        Nr = 0
            
    return Nr

In [9]:
def getALICIAurls(soupObj):
    ALICIAurls = soupObj.find_all('a', href=lambda href: href and href.startswith('/vufind/Record/'))
    
    href_values = []
    
    #Iterar sobre ALICIAurls y extraer el valor de href
    for e in ALICIAurls:
        href = e.get('href').replace("/Save","")
        href_values.append(href)
        
    return href_values

In [10]:
def saveALICIAurlsList(LstObj, kwStr, pathStr):
        
    fName = "ALICIA-" + kwStr + " (" + str(len(LstObj)) + ")" + ".txt"
    fPath = pathStr + fName
    
    with open(fPath, "w") as file:
        for e in LstObj:
            file.write(str(e) + "\n")

# 4. Programa principal para obtener lista de URLs de ALICIA

## 4.1. Abrir navegador

In [11]:
WebNavObj = openWebNav()
WebNavObj.maximize_window()
WebNavObj.get(ALICIA_url)

## 4.2. Lanzando búsquedas

In [12]:
kwList = ["aerogenerador", "energía eólica", "eólica", "generador eólico", "turbina eólica", "wind energy", "wind turbine"]

In [13]:
for kwStr in kwList:
    #Lanzando búsqueda
    runALICIAsearch(WebNavObj, kwStr)
    
    #Obteniendo el número de resultados
    Nr = getNrALICIA(WebNavObj)
    print(f"\tLa búsqueda del término {kwStr} arrojó {Nr} documentos.")
    
    nTabs = int(Nr/20)
    
    ALICIA_URLS = []
    
    for t in range(1, nTabs+2, 1):
        soupObj = getCurrentHTML(WebNavObj)
        tmpURLs = getALICIAurls(soupObj)
        ALICIA_URLS.extend(tmpURLs)
        
        #Visitando la siguiente pestaña
        time.sleep(1)
        try:
            sigBut = WebNavObj.find_element_by_link_text("Siguiente")
            sigBut.click()
            time.sleep(1)
        except:
            print("\t\tEl botón 'Siguiente' esta deshabilitado.")
    
    #Guardando la lista de enlaces
    ALICIA_URLS = list(set(ALICIA_URLS))
    saveALICIAurlsList(ALICIA_URLS, kwStr, './lists/ALICIA/')  

	La búsqueda del término aerogenerador arrojó 220 documentos.
		El botón 'Siguiente' esta deshabilitado.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término energía eólica arrojó 384 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término eólica arrojó 476 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término generador eólico arrojó 63 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término turbina eólica arrojó 90 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término wind energy arrojó 164 documentos.
		El botón 'Siguiente' esta deshabilitado.
	La búsqueda del término wind turbine arrojó 55 documentos.
		El botón 'Siguiente' esta deshabilitado.


## 4.3. Cerrar navegador

In [14]:
WebNavObj.close()
WebNavObj.quit()

#--> ====================================================================================

In [15]:
input("ATENCIÓN!!  Leer mensaje...")

ATENCIÓN!!  Leer mensaje... 


' '

No ejecutar el código lineas abajo si la carpeta '.\lists\ALICIA' esta vacía.

# 5. Programa principal para obtener detalles a partir de listado de URLs de ALICIA

## 5.1. Definición de funciones (II)

In [16]:
def consolidarALICIAtxt(pathStr):
    files = [f for f in os.listdir(pathStr) if re.match(r'ALICIA.*\.txt', f)]
    
    ALICIAurls = set()
    
    for txtf in files:
        with open(os.path.join(pathStr, txtf), 'r') as f:
            for linea in f:
                ALICIAurls.add(linea.strip())
                
    return list(ALICIAurls)

In [17]:
def getPageALICIAMtdata(WebNavObj, urlStr):
    soupObj = getCurrentHTML(WebNavObj)
    
    #Obteniendo el título
    Title = getTitle(soupObj)
       
    #Encontrando los demas metadatos
    metadata_dict = {'Título:':Title,
                     'URL:':urlStr}
    
    #Obteniendo los demas metadatos
    try:
        TableElem = soupObj.find('tbody')
        tmp_dict = getTableMtdataInfo(TableElem)
        metadata_dict.update(tmp_dict)
    except Exception as e:
        print(e)
        print("Error al obtener los datos de la página")    

    return metadata_dict             

In [18]:
def getTitle(soupObj):
    #Encontrando el título    
    try:
        TitElem = soupObj.find('h1', {'property': 'name'})
        if TitElem is None:
            raise Exception('No se pudo encontrar el elemento de título')
        else:
            Title = TitElem.get_text()
    except Exception as e:
        print(e)
        return None
    
    return Title

In [19]:
def getTableMtdataInfo(TableBsObj):
    rowElem = TableBsObj.find_all('tr')
    
    metadata_dict = {}
    for i in range(len(rowElem)):
        labels = rowElem[i].find('th').get_text(strip = True)
        
        if labels == 'Autores:':
            author_tags = rowElem[i].find_all('a')
            authors = []
            
            for tag in author_tags:
                authors.append(tag.text.strip())
            
            metadata_dict['Autor:'] = ';'.join(authors)   
        else:
            values = rowElem[i].find('td').get_text(strip = True)
            metadata_dict[labels] =  values

        
    return metadata_dict

In [20]:
def dictArrtoDf(dictArrObj):
    # obtener lista de claves únicas
    headers = list(set().union(*dictArrObj))
    
    # crear DataFrame con claves como columnas
    df = pd.DataFrame(columns = headers)
    
    # agregar cada diccionario como fila en el DataFrame
    for dic in dictArrObj:
        # Se convierte el diccionario en un DataFrame
        df_temp = pd.DataFrame.from_dict(dic, orient='index').T
        
        # Se concatena el DataFrame temporal con el DataFrame acumulado
        df = pd.concat([df, df_temp], ignore_index=True, sort=False)
    
    return df  

## 5.2. Cargando datos de los enlaces

In [21]:
ALICIAurls = consolidarALICIAtxt('./lists/ALICIA/')
print(f'Se cargaron un total de {len(ALICIAurls)} enlaces para analizar.')

Se cargaron un total de 639 enlaces para analizar.


## 5.3. Abrir navegador

In [22]:
WebNavObj = openWebNav()
WebNavObj.maximize_window()
WebNavObj.get(ALICIA_url)

Esperar que cargue la pagina, puede demorar un poco

## 5.4. Visitando páginas y recolectando información generando un arreglo de diccionarios

In [23]:
ALICIAMtdata = []
for i in range(len(ALICIAurls)): #
    WebNavObj.get(ALICIA_url + ALICIAurls[i])
    time.sleep(0.5)
    
    urlMtdata = getPageALICIAMtdata(WebNavObj, ALICIA_url + ALICIAurls[i])
    
    ALICIAMtdata.append(urlMtdata)

La ejecución de esta celda puede demorar dependiendo de cuantos registros tenga que visitar.

## 5.5. Generando un dataframe a partir del arreglo de diccionarios

In [24]:
ALICIArecords = dictArrtoDf(ALICIAMtdata)

In [25]:
ALICIArecords.to_excel('ALICIA_RawData.xlsx', index = False)

### 5.5.1. Combinando columnas 'Autor:' y 'Autores:'

In [26]:
#ALICIArecords['Autor (es):'] = ALICIArecords['Autor:']
#ALICIArecords['Autor (es):'] = ALICIArecords['Autor (es):'].fillna(ALICIArecords['Autores:'])

In [27]:
#ALICIArecords = ALICIArecords.drop(['Autor:', 'Autores:'], axis=1)

### 5.5.2. Extrayendo un dataframe solo con las columnas mas relevantes

In [28]:
ColRed = ['URL:',
          'Enlace del recurso:',
          'Formato:',
          'Repositorio:', 
          'Institución:',          
          'Título:',
          'Autor:',
          'Fecha de Publicación:', 
         ]

In [29]:
ALICIArecordsRed = ALICIArecords.loc[:, ColRed]

In [30]:
ALICIArecordsRed.to_excel('ALICIA_reduced.xlsx', index = False)

In [31]:
ALICIArecordsRed.columns

Index(['URL:', 'Enlace del recurso:', 'Formato:', 'Repositorio:',
       'Institución:', 'Título:', 'Autor:', 'Fecha de Publicación:'],
      dtype='object')

# 6. Programa principal para normalizar titulo y autores y generar un identificador único para detectar registros dusplicados

In [32]:
import hashlib

## 6.1. Definición de funciones

In [33]:
def normalize_title(title):
    # Eliminar signos de puntuación y espacios en blanco extra
    normalized_title = title.strip().replace(' ','').replace('"','').replace('.', '').replace(',', '').replace('-', '').lower()
    
    return normalized_title

In [34]:
def normalize_authors(authors):
    # Eliminar signos de puntuación y convertir todo a minúsculas
    normalized_authors = authors.lower().replace(',', '').replace('.', '').replace(' ','').replace('-', '')
    
    # Dividir los nombres de los autores y ordenarlos alfabéticamente
    author_list = sorted(normalized_authors.split(';'))
    
    # Unir los nombres de los autores separados por un punto y coma
    normalized_authors = ';'.join(author_list)
    return normalized_authors

In [35]:
def generate_unique_id(title, authors):
    # Normalizar la columna TITULO y la columna AUTORES
    normalized_title = normalize_title(title)
    normalized_authors = normalize_authors(authors)
    
    # Generar un código único solo para el título
    title_id = hashlib.md5(normalized_title.encode()).hexdigest()
    
    # Generar un código único solo para los autores
    authors_id = hashlib.md5(normalized_authors.encode()).hexdigest()
    
    # Concatenar la columna TITULO y la columna AUTORES en una sola cadena
    combined_string = normalized_title + '|' + normalized_authors
    
    # Generar un código único para el registro completo
    unique_id = hashlib.md5(combined_string.encode()).hexdigest()
    return title_id, authors_id, unique_id

## 6.2. Generando identificadores únicos para el titulo, autores y combinación de ambos

In [36]:
#-->
#ALICIArecordsRed = pd.read_excel(r'.\_results\ALICIA_reduced.xlsx')

In [37]:
ALICIArecordsRed['UID_Tit'], ALICIArecordsRed['UID_Aut'], ALICIArecordsRed['UID'] = zip(*ALICIArecordsRed.apply(lambda row: generate_unique_id(row['Título:'], row['Autor:']), axis=1))

In [38]:
ALICIArecordsRed.head()

,URL:,Enlace del recurso:,Formato:,Repositorio:,Institución:,Título:,Autor:,Fecha de Publicación:,UID_Tit,UID_Aut,UID
0,https://alicia.concytec.gob.pe/vufind/Record/0...,http://www.revistas.uni.edu.pe/index.php/tecni...,artículo,Revista UNI - Tecnia,Universidad Nacional de Ingeniería,DISEÑO Y CONSTRUCCIÓN DE UN AEROGENERADOR DE 5...,"González, Salomé;Baldera Teodoro, José Chiroque",2006,2cd59c1556b51921f1d27d83142533f9,3a382ef61f189bd8095e4f4418215745,2ff4461eab123ed6fb6d4535342e922a
1,https://alicia.concytec.gob.pe/vufind/Record/U...,http://hdl.handle.net/20.500.14076/13053,informe técnico,UNI-Tesis,Universidad Nacional de Ingeniería,Construcción de un prototipo de generador eóli...,"Cárdenas Aquino, Pablo Gustavo",2013,2aed701034c85bbcb99eb5a0849fba86,bb6f09938797bff6e6478ad2146f0332,637a0223a00e513c13b96ce0c2603b02
2,https://alicia.concytec.gob.pe/vufind/Record/R...,http://revistas.sqperu.org.pe/index.php/revist...,artículo,Revista de la Sociedad Química del Perú,Sociedad Química del Perú,"Contaminación ambiental , respeto a la socieda...","Guerra Carvallo, Claver Hugo",2021,981b06fcaa5eab2d55794a6da2221fcd,b26390425aa89ddf303d4c993998c55e,a6971755ce8d32768c9a2601df37bd55
3,https://alicia.concytec.gob.pe/vufind/Record/U...,http://hdl.handle.net/20.500.14076/908,tesis de grado,UNI-Tesis,Universidad Nacional de Ingeniería,Estudio de prefactibilidad de la microcentral ...,"Cáceres Vergara, Julio Andrés",2006,32674473a5573d435d6cc43430c9c73a,068304f3360281020f9cc3e491db2247,0e97818b15033f59af2f1abf5a722021
4,https://alicia.concytec.gob.pe/vufind/Record/A...,https://hdl.handle.net/20.500.12543/4579,informe técnico,ANA-Institucional,Autoridad Nacional del Agua,Estudio de suelos en los Centros Culturales Pi...,Corporación de Fomento Económico y Social de L...,1969,b9ed22bc0993efb65ddebbd54d2df8aa,c4e64e5aba88bc4d0da7a9880260bd6d,6aa2b2e838cb5089bcafc3c86c1c050d


In [39]:
ALICIArecordsRed.to_excel('ALICIA_reduced_UID.xlsx', index = False)

## 6.3. Cerrar navegador

In [40]:
WebNavObj.close()
WebNavObj.quit()